<a href="https://colab.research.google.com/github/chelsk5/UW-LING-573-Group-5/blob/main/LING573_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Authors: Nina Koh, Chelsea Kendrick
# Install dependencies
# !pip install --quiet --upgrade pip
# !pip install --quiet --upgrade huggingface_hub
# !pip install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --upgrade
# !pip install --quiet --upgrade torchao
# !pip install --quiet --upgrade accelerate
# !pip install --quiet --upgrade transformers
# !pip install --quiet sentence-transformers==2.2.2 fsspec==2023.6.0 numpy==1.26.4
# !pip install --quiet --upgrade transformers huggingface_hub sentence-transformers fsspec numpy rouge-metric nltk

# !pip install torch==2.0.1+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install torch==2.1.2+cu118 torchvision==0.16.2+cu118 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118
# !pip install --quiet sentence-transformers==2.2.2 transformers==4.30.2 \
#  fsspec==2023.6.0 numpy==1.26.4 numba>=0.60.0
# !pip install --upgrade transformers huggingface-hub

# !pip install torch==2.0.1 transformers==4.37.2 accelerate==0.25.0 huggingface-hub==0.20.3

!pip uninstall -y transformers accelerate huggingface-hub torch torchvision torchaudio \
  tokenizers sentence-transformers peft fastai timm diffusers
!pip install torch==2.1.2+cu118 torchvision==0.16.2+cu118 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.38.2 accelerate==0.25.0 huggingface-hub==0.27.1
!pip install datasets rouge-metric nltk

import nltk
nltk.download('punkt')

# Don't use requirements.txt unless working in a local environment
# !pip install -r UW_LING_573_Group_5/requirements.txt

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
Found existing installation: huggingface-hub 0.27.1
Uninstalling huggingface-hub-0.27.1:
  Successfully uninstalled huggingface-hub-0.27.1
Found existing installation: torch 2.1.2+cu118
Uninstalling torch-2.1.2+cu118:
  Successfully uninstalled torch-2.1.2+cu118
Found existing installation: torchvision 0.16.2+cu118
Uninstalling torchvision-0.16.2+cu118:
  Successfully uninstalled torchvision-0.16.2+cu118
Found existing installation: torchaudio 2.1.2+cu118
Uninstalling torchaudio-2.1.2+cu118:
  Successfully uninstalled torchaudio-2.1.2+cu118
Found existing installation: tokenizers 0.15.2
Uninstalling tokenizers-0.15.2:
  Successfully uninstalled tokenizers-0.15.2
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached h

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Clone repo & set up environment

!rm -rf UW-LING-573-Group-5
!git clone https://github.com/chelsk5/UW-LING-573-Group-5.git
!mv UW-LING-573-Group-5 UW_LING_573_Group_5 # Python modules use underscores

Cloning into 'UW-LING-573-Group-5'...
remote: Enumerating objects: 519, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 519 (delta 33), reused 32 (delta 17), pack-reused 457 (from 1)
Receiving objects: 100% (519/519), 391.17 KiB | 16.30 MiB/s, done.
Resolving deltas: 100% (99/99), done.
mv: cannot move 'UW-LING-573-Group-5' to 'UW_LING_573_Group_5/UW-LING-573-Group-5': Directory not empty


In [4]:
%cd UW_LING_573_Group_5/
%ls

/content/UW_LING_573_Group_5
LING573_Model.ipynb  requirements.txt  summaries.json
Minutes.md           rouge_eval.py     tf-idf.py
README.md            rouge.py          UW-LING-573-Group-5/


In [5]:
# 1. Load data
from datasets import load_dataset
opinosis = load_dataset("kavgan/opinosis", split="train") # HuggingFace hub
dataset = opinosis.train_test_split(test_size=0.2,seed=1) # follow common split ratio
train_data = dataset["train"]
test_data = dataset["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# 2. Preprocessing
def clean_data(sent_to_sum):
  """Cleans the given review and returns a dictionary
  of aggregated reviews and the first gold summary"""
  reviews = sent_to_sum["review_sents"].strip()
  # Choose the first gold summary
  gold_summary = sent_to_sum["summaries"][0].strip()
  return {
      "reviews": reviews,
      "gold_summary": gold_summary
  }

# Sanity check!
# Apply cleaning to a small sample
cleaned_train = train_data.map(clean_data)
cleaned_test = test_data.map(clean_data)

# Print the result
print("Input reviews:")
print(cleaned_train[0]["reviews"])
print("\nTarget summary:")
print(cleaned_train[0]["gold_summary"])

print("\nInput reviews:")
print(cleaned_train[1]["reviews"])
print("\nTarget summary:")
print(cleaned_train[1]["gold_summary"])

Input reviews:
The food for our event was delicious .
 The food in the lounge was great and very fresh, , , salads, sandwiches etc .
 As far as food, walk a few blocks toward Michigan Ave turn left or right and there are plently of less expensive places to eat .
 The Palm resturant in the hotel had some specials Sunday night, we ate there and the food service,etc were outstanding portions are large and we shared since we are not big eaters .
 Took the charge of the minibar which we had used to keep my 2 year old sons food .
 We never ate anything onsite so I can't vouch for the food options immediately avialable .
 The Lobby bar does not serve food very late at night and we couldn't find any vending machines for soda or snacks, so stop at a nearby market before going in for the night .
 In any case, I had allotted approximately $150 for food for 4 days 3 nights .
 But the presentation was awesome and the food was so good and not having to walk in the dark at night and scare my mother

In [7]:
# 3. Load tokenizer & model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype="float16",
#     bnb_4bit_quant_type="nf4",  # You can also use "fp4" depending on your needs
#     bnb_4bit_use_double_quant=True
# )

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# 4. Tokenize reviews & gold summaries
def tokenize_data(cleaned_data):
  """Convert the raw text into token IDs for model processing"""
  # Prepend task for T5 to execute
  pref_text = "summarize: " + cleaned_data["reviews"]

  # Tokenize input (reviews)
  # Ensure that text does not exceed (is in fact exactly) maximum of 512 tokens
  tokenized_input = tokenizer(pref_text, max_length=512, truncation=True, padding="max_length")

  # Tokenize target output (gold summary)
  gold_summary = cleaned_data["gold_summary"]
  labels = tokenizer([gold_summary], max_length=64, truncation=True) # expects a list

  # Add target labels to the model input
  # Format for HuggingFace Trainer
  tokenized_input["labels"] = labels["input_ids"][0] # get first (& only) item of nested list
  return tokenized_input

#apply tokenization to data
tokenized_train = cleaned_train.map(tokenize_data, batched=False) # one row at a time
tokenized_test = cleaned_test.map(tokenize_data, batched=False)

#check tokenization
print("Tokenized input example (input_ids):")
print(tokenized_train[0]["input_ids"])
print("Tokenized target (labels):")
print(tokenized_train[0]["labels"])

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Tokenized input example (input_ids):
[21603, 10, 37, 542, 21, 69, 605, 47, 3326, 3, 5, 37, 542, 16, 8, 9658, 47, 248, 11, 182, 1434, 6, 3, 6, 3, 6, 5870, 7, 6, 20017, 672, 3, 5, 282, 623, 38, 542, 6, 1482, 3, 9, 360, 6438, 2957, 5847, 8945, 919, 646, 42, 269, 11, 132, 33, 4752, 295, 120, 13, 705, 2881, 1747, 12, 3, 1544, 3, 5, 37, 8005, 880, 450, 288, 16, 8, 1595, 141, 128, 534, 7, 1771, 706, 6, 62, 3, 342, 132, 11, 8, 542, 313, 6, 21117, 130, 4590, 17622, 33, 508, 11, 62, 2471, 437, 62, 33, 59, 600, 23852, 7, 3, 5, 304, 1825, 8, 1567, 13, 8, 3016, 1047, 84, 62, 141, 261, 12, 453, 82, 204, 215, 625, 520, 7, 542, 3, 5, 101, 470, 3, 342, 959, 3, 24452, 78, 27, 54, 31, 17, 3, 1621, 2295, 21, 8, 542, 931, 2017, 3, 2960, 138, 179, 3, 5, 37, 17464, 969, 1207, 405, 59, 1716, 542, 182, 1480, 44, 706, 11, 62, 2654, 31, 17, 253, 136, 19518, 53, 4096, 21, 12495, 42, 12751, 6, 78, 1190, 44, 3, 9, 4676, 512, 274, 352, 16, 21, 8, 706, 3, 5, 86, 136, 495, 6, 27, 141, 66, 14708, 26, 3241, 25689, 21, 3

In [9]:
# 5. Create batch of examples
from transformers import DataCollatorForSeq2Seq

# Dynamically pad sentences to the longest length
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [24]:
# 6. Evaluate model using ROUGE
!pip install rouge-score evaluate
import numpy as np
from UW_LING_573_Group_5 import rouge
from rouge import rouge_scorer

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

def compute_rouge(eval_pred):
  """Decodes token IDs and computes the ROUGE scores of generated summaries"""
  predictions, labels = eval_pred # model outputs & gold summaries as token IDs
  # Decode model output IDs back into text
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

  # Replace padding tokens IDs (-100) with tokenizer's pad token ID (usually 0)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Decode gold summary IDs back into text
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Compute precision, recall, & f1 across ROUGE scores
  rouge1_p, rouge1_r, rouge1_f = [], [], []
  rouge2_p, rouge2_r, rouge2_f = [], [], []
  rougeL_p, rougeL_r, rougeL_f = [], [], []

  for pred, label in zip(decoded_preds, decoded_labels):
    scores = scorer.score(pred, label)
    # ROUGE-1
    rouge1_p.append(scores["rouge1"].precision)
    rouge1_r.append(scores["rouge1"].recall)
    rouge1_f.append(scores["rouge1"].fmeasure)

    # ROUGE-2
    rouge2_p.append(scores["rouge2"].precision)
    rouge2_r.append(scores["rouge2"].recall)
    rouge2_f.append(scores["rouge2"].fmeasure)

    # ROUGE-L
    rougeL_p.append(scores["rougeL"].precision)
    rougeL_r.append(scores["rougeL"].recall)
    rougeL_f.append(scores["rougeL"].fmeasure)

  print("\n--- ORIGINAL ---")
  print("Prediction:", decoded_preds[0])
  print("Gold:", decoded_labels[0])

  print("\n--- HF-style Preprocessed ---")
  print("Pred:", preprocess(decoded_preds[0]))
  print("Gold:", preprocess(decoded_labels[0]))

  # Return rounded mean across scores
  return {
      "rouge1_precision": round(np.mean(rouge1_p), 4),
      "rouge1_recall": round(np.mean(rouge1_r), 4),
      "rouge1_f1": round(np.mean(rouge1_f), 4),
      "rouge2_precision": round(np.mean(rouge2_p), 4),
      "rouge2_recall": round(np.mean(rouge2_r), 4),
      "rouge2_f1": round(np.mean(rouge2_f), 4),
      "rougeL_precision": round(np.mean(rougeL_p), 4),
      "rougeL_recall": round(np.mean(rougeL_r), 4),
      "rougeL_f1": round(np.mean(rougeL_f), 4),
  }

# The code below is only useable if this ROUGE runtime error gets resolved:
# https://huggingface.co/evaluate-metric?sort_spaces=modified#spaces

# import evaluate

# rouge = evaluate.load("rouge")

# def compute_rouge(eval_pred):
#   """Decodes token IDs and computes the ROUGE score of generated summaries"""
#   predictions, labels = eval_pred # model outputs & gold summaries as token IDs
#   # Decode model output IDs back into text
#   decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

#   # Replace padding tokens IDs (-100) with tokenizer's pad token ID (usually 0)
#   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#   # Decode gold summary IDs back into text
#   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#   # Compare model output against gold labels via ROUGE
#   result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#   clean_result = {k: round(v, 4) for k,v in result.items()} # map rouge type to value
#   return clean_result

In [19]:
# 6B. Alternative ROUGE evaluation using rouge_metric PyRouge
import numpy as np
from rouge_metric import PyRouge
import re
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

rouge = PyRouge(
    rouge_n=(1, 2, 4),
    rouge_l=True,
    rouge_w=True,
    rouge_s=True,
    rouge_su=True,
    skip_gap=4
)

def preprocess(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  tokens = text.split()
  stemmed = [stemmer.stem(t) for t in tokens]
  return ' '.join(stemmed)
  # return text.strip() # apply minimal preprocessing to match HF

def compute_rouge(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  hypotheses = [preprocess(pred) for pred in decoded_preds]
  references = [[preprocess(label)] for label in decoded_labels]

  #scores = rouge.get_scores(hypotheses, references, avg=True)
  scores = rouge.evaluate_tokenized(hypotheses, references)

  print("\n--- ORIGINAL ---")
  print("Prediction:", decoded_preds[0])
  print("Gold:", decoded_labels[0])

  print("\n--- PyROUGE-style Preprocessed ---")
  print("Pred:", preprocess(decoded_preds[0]))
  print("Gold:", preprocess(decoded_labels[0]))

  return {
      "rouge1_precision": round(scores["rouge-1"]["p"], 4),
      "rouge1_recall": round(scores["rouge-1"]["r"], 4),
      "rouge1_f1": round(scores["rouge-1"]["f"], 4),
      "rouge2_precision": round(scores["rouge-2"]["p"], 4),
      "rouge2_recall": round(scores["rouge-2"]["r"], 4),
      "rouge2_f1": round(scores["rouge-2"]["f"], 4),
      "rougeL_precision": round(scores["rouge-l"]["p"], 4),
      "rougeL_recall": round(scores["rouge-l"]["r"], 4),
      "rougeL_f1": round(scores["rouge-l"]["f"], 4),
      "rougeSU4_precision": round(scores["rouge-su4"]["p"], 4),
      "rougeSU4_recall": round(scores["rouge-su4"]["r"], 4),
      "rougeSU4_f1": round(scores["rouge-su4"]["f"], 4),
  }

In [25]:
# 7. Train model
# !pip install --upgrade accelerate==0.27.2

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Manually tune hyperparameters

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-small-results", # folder to save progress
    evaluation_strategy="epoch", # evaluate after one complete pass
    learning_rate=2e-5,
    per_device_train_batch_size=8, # small batch size for small data
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3, # number of checkpoints (subject to change?)
    num_train_epochs=3, # total passes thru data (also stc...)
    predict_with_generate=True,
    fp16=False,# changed to False to avoid ValueError
    report_to="none"
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_rouge
)

# Train (finally)
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1 Precision,Rouge1 Recall,Rouge1 F1,Rouge2 Precision,Rouge2 Recall,Rouge2 F1,Rougel Precision,Rougel Recall,Rougel F1
1,No log,3.350780,0.232100,0.244500,0.222800,0.073400,0.060500,0.062300,0.208700,0.204800,0.194400
2,No log,3.335762,0.232100,0.244500,0.222800,0.073400,0.060500,0.062300,0.208700,0.204800,0.194400
3,No log,3.329674,0.232100,0.241400,0.221300,0.073400,0.059700,0.062000,0.208700,0.202000,0.193000



--- ORIGINAL ---
Prediction: If a case was included, as with the Kindle 1, that would have been reflected
Gold: While the price of the Amazon Kindle 2 is high, it is worth it. That said, beware: the prices of recently released books are high.

--- HF-style Preprocessed ---
Pred: if a case wa includ as with the kindl 1 that would have been reflect
Gold: while the price of the amazon kindl 2 is high it is worth it that said bewar the price of recent releas book are high

--- ORIGINAL ---
Prediction: If a case was included, as with the Kindle 1, that would have been reflected
Gold: While the price of the Amazon Kindle 2 is high, it is worth it. That said, beware: the prices of recently released books are high.

--- HF-style Preprocessed ---
Pred: if a case wa includ as with the kindl 1 that would have been reflect
Gold: while the price of the amazon kindl 2 is high it is worth it that said bewar the price of recent releas book are high

--- ORIGINAL ---
Prediction: If a case was included

TrainOutput(global_step=15, training_loss=2.952422332763672, metrics={'train_runtime': 11.7984, 'train_samples_per_second': 10.171, 'train_steps_per_second': 1.271, 'total_flos': 16241016176640.0, 'train_loss': 2.952422332763672, 'epoch': 3.0})

In [ ]:
# 8. Log metrics to JSON file
metrics = trainer.evaluate()
print(metrics) # sanity check

import json
with open("t5-small-results/metrics.json", "w") as f:
  json.dump(metrics, f, indent=2)

{'eval_loss': 3.914048910140991, 'eval_rouge1_precision': 0.7281, 'eval_rouge1_recall': 0.71, 'eval_rouge1_f1': 0.7189, 'eval_rouge2_precision': 0.4209, 'eval_rouge2_recall': 0.386, 'eval_rouge2_f1': 0.4027, 'eval_rougeL_precision': 0.7281, 'eval_rougeL_recall': 0.71, 'eval_rougeL_f1': 0.7189, 'eval_rougeSU4_precision': 0.5528, 'eval_rougeSU4_recall': 0.5251, 'eval_rougeSU4_f1': 0.5386, 'eval_runtime': 15.7814, 'eval_samples_per_second': 0.697, 'eval_steps_per_second': 0.127, 'epoch': 3.0}


In [ ]:
# 10. Save predictions & labels in separate files
import os
system_dir = "t5-small-results/system_summaries" # t5 predictions
model_dir = "t5-small-results/model_summaries" # gold reference
os.makedirs(system_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

# Maybe this can be streamlined later since it's handled by compute_rouge()...
predictions = trainer.predict(tokenized_test)
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)
labels = np.where(predictions.label_ids != -100, predictions.label_ids, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

for i, (pred, ref) in enumerate(zip(decoded_preds, decoded_labels)):
    with open(f"{system_dir}/{i}.txt", "w") as sys_f:
        sys_f.write(pred.strip() + "\n")
    with open(f"{model_dir}/{i}.A.1.txt", "w") as ref_f:
        ref_f.write(ref.strip() + "\n")